# Genre Sorter

## Set Up
1. Create `.env` file

``` bash
  SPOTIFY_CLIENT_ID=
  SPOTIFY_CLIENT_SECRET=
  SPOTIFY_USERNAME=
  SPOTIFY_PASSWORD=
```

2. Run all cells


In [40]:
%pip install spotipy python-dotenv openai tqdm

%load_ext dotenv
%dotenv


[notice] A new release of pip available: 22.3.1 -> 24.1.2
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [19]:
%reload_ext dotenv

In [6]:
import os
import spotipy
from spotipy.oauth2 import SpotifyOAuth

scope = "user-library-read"

# sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
sp_oauth = SpotifyOAuth(scope=scope, client_id=os.getenv("SPOTIPY_CLIENT_ID"), client_secret=os.getenv(
    "SPOTIPY_CLIENT_SECRET"), redirect_uri=os.getenv("SPOTIPY_REDIRECT_URI"))

# Get the authorization URL
auth_url = sp_oauth.get_authorize_url()

print(f"Please navigate to this URL to authorize the application: {auth_url}")

# After authorization, Spotify will redirect to the redirect URI with a code parameter
response_url = input("Enter the URL you were redirected to: ")

# Parse the code from the URL
code = sp_oauth.parse_response_code(response_url)

# Get the access token
token_info = sp_oauth.get_access_token(code)

# Create a new Spotify object with the access token
sp = spotipy.Spotify(auth=token_info["access_token"])

Please navigate to this URL to authorize the application: https://accounts.spotify.com/authorize?client_id=c170e034b76346b9ac1f8cb7690e82f3&response_type=code&redirect_uri=https%3A%2F%2Felizabethfrey.me%2F&scope=user-library-read


/var/folders/dm/l18067jn62qb3fkpng93sqhr0000gn/T/ipykernel_60164/76519990.py:23: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token_info = sp_oauth.get_access_token(code)


In [7]:
def get_spotify_client():
    token_info = sp_oauth.get_cached_token()
    if not token_info:
        auth_url = sp_oauth.get_authorize_url()
        print(
            f"Please navigate to this URL to authorize the application: {auth_url}")

        token_info = sp_oauth.get_access_token(code)

    if sp_oauth.is_token_expired(token_info):
        token_info = sp_oauth.refresh_access_token(token_info['refresh_token'])

    return spotipy.Spotify(auth=token_info['access_token'])

In [38]:
# Fetch user's saved tracks
def get_all_saved_tracks(sp):
    all_tracks = []
    all_artist_genres = {}
    all_track_names = []

    results = sp.current_user_saved_tracks(limit=50)
    all_artist_genres, all_track_names = fetch_track_and_artist_info(
        results['items'], all_artist_genres, all_track_names)
    all_tracks.extend(results['items'])

    while results['next']:
        results = sp.next(results)
        all_artist_genres, all_track_names = fetch_track_and_artist_info(
            results['items'], all_artist_genres, all_track_names)
        all_tracks.extend(results['items'])

    return all_tracks, all_artist_genres, all_track_names

# Fetch artist genres


def fetch_artist_genres(sp, artist_ids, artist_genres):
    artists = sp.artists(artist_ids)
    for artist in artists['artists']:
        artist_genres[artist['name']] = artist['genres']
    return artist_genres

# Get artist IDs and track names


def fetch_track_and_artist_info(tracks, artist_genres, track_names):
    artist_ids = []
    for item in tracks:
        track = item['track']
        artist_ids.append(track['artists'][0]['id'])
        track_names.append(track['name'])

    artist_genres = fetch_artist_genres(sp, artist_ids, artist_genres)
    return artist_genres, track_names


sp = get_spotify_client()

tracks, artist_genres, track_names = get_all_saved_tracks(sp)

In [89]:
track_name_dict = {}
for i, track in enumerate(track_names):
    track_name_dict[track] = tracks[i]

track_id_dict = {}
for i, track in enumerate(tracks):
    track_id_dict[track['track']['id']] = track
track_id_dict.keys()

dict_keys(['2gSVKq4fs4rYE6ukeKnVWi', '6KqM3xmPIDonsTjCSGrrr5', '09RamrocZneYcLYLRIxYha', '3Kwj07sN1wdzlRNMU8rNX0', '22hOKPKTAegLoLJKM33K4a', '5hXEcqQhEjfZdbIZLO8mf2', '5rzHkSHRyZUdQcCX6noFDB', '4eAV1ZCliRotEcjzEbOWNW', '561pBFcFL2Pwb9HPO9tU8J', '7AvuIMRKdo5RvS6YIzXjsj', '6rqUOLt9dpfMANLMKTzoWj', '1akuYztpg4VfZT8Lg4N1w9', '68O2VV2eloAmNFjI4fEaz0', '331l3xABO0HMr1Kkyh2LZq', '4eamJr2k9YDH8j58RSTmSc', '1dcK8O2qEq7E34o4ImQEco', '5b5cPscqVEMChvDqscVw26', '7b6sWLSnbZhSIeqNGrJ4py', '1awQLv1J6LkeEQ5co4OOoQ', '4hGLX2kT2Ayrw0Uh2oQp0J', '68R0zVUeMJ2C852Ov6d2Mh', '5Hh09KHzQ9NRjRd6HIes5R', '0NLebGDyFekhMcQKG7xfaP', '7221xIgOnuakPdLqT0F3nP', '5crY7axm3GejOxy5PsnnPk', '6WiGRFQfx2qw49MA7OeDYK', '5iwUwADk4uQUvgMF6PlTRl', '2Nq4SFbvYYZa8AF7lD7CWU', '49fSj7bWWBfsi8thBoHN3S', '3NYm3ZimuJNb0Z0px5OEwO', '7qMMw7ngQEtW4Cam5GASW3', '2NvYdollt2CTUJYn66OBt9', '4nyF5lmSziBAt7ESAUjpbx', '1ZZaOAx86pHrZR7Kb0RbKM', '5lE2EFXt4muvLFMGQg4hZN', '0B4U9FT1WaFwUiiCPOAak7', '3CqaTHbiU2nBy3Ar9RnDHN', '3Y1EvIgEVw51XtgNEgpz5c', '

In [96]:

import json
from openai import OpenAI
from tqdm import tqdm

playlist_genres = [
    "Upbeat Pop",
    "Energetic Hip Hop",
    "Relaxed Jazz",
    "Reflective Classical",
    "Vibrant EDM",
    "Groovy House",
    "Bold Rock",
    "Smooth Jazz",
    "Danceable Pop",
    "Futuristic EDM",
    "Chill House",
    "Indie Rock",
    "Quirky Indie",
    "Upbeat Hip Hop",
    "Intense Classical",
    "Lively Jazz",
    "Intense EDM",
    "Pulsing House"
]

genre_playlists = {genre: [] for genre in playlist_genres}


def sort_tracks(tracks):
    # Prepare the prompt for GPT-4
    prompt = "Here are the songs, with their track id and associated genres and artists:"
    for i, track in enumerate(tracks):
        artist_name = track['track']['artists'][0]['name']
        track_id = track['track']['id']
        track_name = track['track']['name']
        genres = ', '.join(artist_genres[artist_name])
        prompt += f"Track: {track_name}, Track_Id: {
            track_id}, Artist: {artist_name}, Genres: {genres}\n"

    prompt += "\n".join([f"Genre: {genre}" for genre in playlist_genres])
    prompt += "\n"

    # Call GPT-4
    client = OpenAI(
        # This is the default and can be omitted
        api_key=os.environ.get("OPENAI_API_KEY"),
    )

    chat_completion = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": "You are a simple sorting robot for songs on spotify. I will provide you with a list of songs with their artists and genres, as well as a list of categories by genre to sort the songs into. Your job is simply to look at each song and the associated artist genre and sort the songs into the correct categories. You are to respond only with a valid JSON object with the following schema: {\"genre1\": [\"track_id1\", \"track_id2\"], \"genre2\": [\"track_id3\", \"track_id4\"]}. An example response would be {\"pop\": [<track_id1>, track_id2>], \"rock\": [<track_id3>, <track_id4>]}. \n Do not return any exmaple or dummy data-- you should return the exact same number of songs that I provide you, and your sole task is to copy the track ids IDENTICALLY into the correct genre categoreies."
            },
            {
                "role": "user",
                "content": prompt,
            }
        ],
    )

    chat_completion = chat_completion.choices[0].message.content

    return chat_completion

In [105]:
all_track_ids = [track['track']['id'] for track in tracks]
found_track_ids = []

for i in tqdm(range(0, len(all_track_ids), 50)):
    print('Batch ', i+1, ' to ', i+50)
    json_output = sort_tracks(tracks[i:i+50])
    parsed_json_output = json.loads(json_output)
    for genre, track_ids in parsed_json_output.items():
        if genre in genre_playlists:
            for track_id in track_ids:
                if track_id in track_id_dict:
                    track = track_id_dict[track_id]
                    genre_playlists[genre].append(track)
                    found_track_ids.append(track_id)
                else:
                    print(f"Track with id {
                          track_id} not found in saved tracks")
        else:
            print(f"Genre {genre} not found in playlist genres")

missing_track_ids = set(all_track_ids) - set(found_track_ids)

  0%|          | 0/38 [00:00<?, ?it/s]

Batch  1  to  50


  3%|▎         | 1/38 [00:12<07:41, 12.48s/it]

Track with id 4e0qvvtZmKPMCU13FieAos not found in saved tracks
Track with id 2GgvtfWigGGshnN7nGhW8x not found in saved tracks
Track with id 3HZnLCP9Jl9JWzMQkMG7jl not found in saved tracks
Track with id 8s2N7yI0MgaoZpw6feXB8i not found in saved tracks
Batch  51  to  100


  5%|▌         | 2/38 [00:32<10:13, 17.03s/it]

...

 95%|█████████▍| 36/38 [12:58<01:05, 32.72s/it]

Track with id 0WgG1m9GUlRFARRHd1FkzX not found in saved tracks
Track with id 03XzT3bKTahz7tC8ZvqAOh not found in saved tracks
Track with id 5xu7nrKiVHslrdlNHRsSsP not found in saved tracks
Track with id 4Z2wrVIUVoeqjfZ6EsWTMw not found in saved tracks
Track with id 52fdCTeaf2Z9MVC0DnwNSB not found in saved tracks
Track with id 4MyE2RW4KHy7mkzAF0KxR7 not found in saved tracks
Track with id 4f4iXgHcuTGudeWgpGhBgi not found in saved tracks
Track with id 5ix6R1RZ8vb3mGyZIKvsi3 not found in saved tracks
Track with id 56ePsbMeDZSJgagLf8Fbnh not found in saved tracks
Track with id 0GNEYHqzOUNVku94H4aGBs not found in saved tracks
Track with id 7FK5J1fgNCMntm3FQxyxJr not found in saved tracks
Track with id 064ZX5yOG77bR2vZTaKJKt not found in saved tracks
Track with id 1SrNTn56OpStUmzXUAo9Zz not found in saved tracks
Track with id 34aBbdHBcHDbaYlBJKBLiw not found in saved tracks
Track with id 2vKhXEP1b4I4VU660fSiFS not found in saved tracks
Track with id 33npCKkvf7WfAbETIzeeRM not found in saved

 97%|█████████▋| 37/38 [13:13<00:27, 27.50s/it]

Batch  1851  to  1900


100%|██████████| 38/38 [13:22<00:00, 21.11s/it]


In [108]:
print(f"Missing track ids: {missing_track_ids}")
missing_tracks = [track_id_dict[track_id] for track_id in missing_track_ids]

while len(missing_tracks) > 0:
    for i in tqdm(range(0, len(missing_tracks), 50)):
        print('Batch ', i+1, ' to ', i+50)
        json_output = sort_tracks(tracks[i:i+50])
        parsed_json_output = json.loads(json_output)
        for genre, track_ids in parsed_json_output.items():
            if genre in genre_playlists:
                for track_id in track_ids:
                    if track_id in track_id_dict:
                        track = track_id_dict[track_id]
                        genre_playlists[genre].append(track)
                        found_track_ids.append(track_id)
                    else:
                        print(f"Track with id {
                              track_id} not found in saved tracks")
            else:
                print(f"Genre {genre} not found in playlist genres")

    missing_track_ids = set(all_track_ids) - set(found_track_ids)
    missing_tracks = [track_id_dict[track_id]
                      for track_id in missing_track_ids]

Missing track ids: {'1TcF5F10EN17vraisG1gQv', '6i6zi5TgXuKUxooOMqhKfQ', '2o9LAypwGH4ctV0i9boo6d', '1QK0cJlXnfq3q7Vo8mVL4w', '62bjfb5YMLvZU44vfjJVxp', '4mH2RJAyMKQpGjyGd2JMDW', '5WkCcSa78lM1Ym4LXzJUiN', '2IAR0DziHCjSu16gR4ihvy', '4fSIb4hdOQ151TILNsSEaF', '4hOyjY9EUksjyylV1QZaF3', '4ZtFanR9U6ndgddUvNcjcG', '7mAyraTz5O6WH06Tt3mLmJ', '14boA065HMHWvqyPPEeN4y', '14YooSSTGCikWjqD0e5r4j', '2tGvwE8GcFKwNdAXMnlbfl', '2279iXhVV3qqY98p0qcAFn', '4lkiVrmAYyPCtW3AFRs14e', '2EcsgXlxz99UMDSPg5T8RF', '5uISqR4lyfgJ0snckYmQpJ', '442j8VxaB60dWf9cBFuX5w', '1NrDot6zcivIjCVMFoe7uV', '5EXD5rXJ4IVb8g4xSwT0fc', '1FavY8gfv2Hr5YDCfT1K7J', '3LBM25IgxI9y549SiYxtSi', '6tCV8RZVP7zW59Ybxj6COL', '6CxdeM5S42kp7GyMlLs1Ea', '5xKJI9aPQhuTdTq8BrJ8fL', '2UIraNrjFYMnu1HI15QR4S', '1yPKJ5d5mDn63uneEBHzTG', '5iwUwADk4uQUvgMF6PlTRl', '6vU3Wqp5PbZbGex8xY1dAW', '4CxFN5zON70B3VOPBYbd6P', '6siDDghVE7fxel5o7fE4OF', '6MRUQNjaUud1TiFBRV8HNq', '2bI6KAUqXeIXGAEEvup8ri', '4VginDwYTP2eaHJzO0QMjG', '7fYRg3CEbk6rNCuzNzMT06', '6e40mgJiCid5HRAGr

  0%|          | 0/18 [00:00<?, ?it/s]

Batch  1  to  50


  6%|▌         | 1/18 [00:11<03:15, 11.49s/it]

Genre Alternative R&B not found in playlist genres
Batch  51  to  100


...

Track with id 69OOfCvizpdlIRAFY not found in saved tracks
Batch  51  to  100


 18%|█▊        | 2/11 [00:24<01:48, 12.01s/it]

Batch  101  to  150


 18%|█▊        | 2/11 [00:27<02:04, 13.79s/it]


RateLimitError: Error code: 429 - {'error': {'message': 'Your account is not active, please check your billing details on our website.', 'type': 'billing_not_active', 'param': None, 'code': 'billing_not_active'}}

In [112]:
# Print titles of missing track ids
missing_track_titles = [track['track']['name'] for track in missing_tracks]
print(f"Missing track titles: {missing_track_titles}")

Missing track titles: ['Come Around Me', 'BEATS FOR THE UNDERGROUND', 'RASTA PASTA (ft. Masego)', 'Nothing', 'Todo De Ti', 'good 4 u', 'Lucinda', 'I don’t even speak spanish lol - feat. Rio Santana, Judah, Andrez Babii', '聽媽媽的話', 'happier', 'Bad Habit', 'Tyler Durden', 'If You Were the Rain', 'Who Do You Call?', 'Boomin (feat. Missy Elliott & Future)', 'let me love you', 'Sweater Weather', 'Hey DJ - DJ Heartbreak Trance Mix', 'Tell Me What You Want', 'Circles', 'Keep On', 'Coffee', 'MIDDLE OF THE NIGHT', 'Cream', 'Indian Summer', 'Feeling Good', 'Immortal', 'HANDSOMER (Remix) (Feat. Ktlyn)', 'We Groovin (with Jazzy)', 'Freed from Desire', 'Neon Guts (feat. Pharrell Williams)', 'Aria', 'Amber', 'Memories', 'Yuck', 'YOSEMITE', 'Lemon Tree', "f**k it, i'm the man", 'Street by Street', 'Brand New Day', 'Reggaetón Lento (Remix)', 'gucci slides (feat. Smino)', 'CHRONICALLY CAUTIOUS', 'Heebiejeebies - Bonus', 'PlantedInMyMind.Memo', 'Chained to the Rhythm', 'atlas', 'Unravel Me', 'Starting Ov

In [115]:
# Convert genre_playlists to json
genre_playlists_json_ids = {genre: [track['track']['id'] for track in tracks]
                        for genre, tracks in genre_playlists.items()}
genre_playlists_json_names = {genre: [track['track']['name'] for track in tracks]
                        for genre, tracks in genre_playlists.items()}
genre_playlists_json = {genre: [track for track in tracks]
                        for genre, tracks in genre_playlists.items()}

# Save genre playlists to a file
with open('genre_playlists_ids.json', 'w') as f:
    json.dump(genre_playlists_json_ids, f)

with open('genre_playlists_names.json', 'w') as f:
    json.dump(genre_playlists_json_names, f)

with open('genre_playlists.json', 'w') as f:
    json.dump(genre_playlists_json, f)